In [91]:
import ast
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

from copy import deepcopy
from sklearn.model_selection import KFold

In [92]:
#modelo MLP, 1 camada intermediára com 500 neurônios
class MLP_500_classifier(nn.Module):
    def __init__(self, input_len):
        super().__init__()

        self.layer1 = nn.Linear(input_len, 500)  
        self.relu = nn.ReLU()                          
        self.layer2 = nn.Linear(500, 30) 

    def forward(self, x):

        x = self.relu(self.layer1(x))  
        logits = self.layer2(x)
        
        return logits

In [93]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

y_best_ansatz = np.argmax(y, axis=1)  

# criando o kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [94]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [95]:
# implementação de um critério de parada para o modelo parar no ponto "ótimo"
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [96]:
# 3. Função para treinar um fold
def train_fold(model, train_loader, val_loader, criterion, optimizer, n_epochs=1000000, patience = 10, min_delta = 0.0):
    model.to(device)
    early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)
    best_model = None
    best_acc = 0.0
    
    for epoch in range(n_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze()  # Converte (batch_size, 1) para (batch_size,)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validação
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            total = 0
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                y_val = y_val.squeeze()  # Garante que y_val é 1D
                
                outputs = model(X_val)
                val_loss += criterion(outputs, y_val).item()  
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == y_val).sum().item()
                total += y_val.size(0)
        
        
        val_acc = correct / total
        val_loss /= len(val_loader)
        train_loss /= len(train_loader)
        
        print(f'Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.4f}')
        
        # Early Stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
        # Salvar melhor modelo
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = deepcopy(model.state_dict())
    
    return best_model, best_acc

In [97]:
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n=== Fold {fold + 1}/{3} ===")
    
    # Dividir e preparar dados
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_best_ansatz[train_idx], y_best_ansatz[val_idx]
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    
    # Inicializar modelo e otimizador
    model = MLP_500_classifier(22)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # L2 regularization
    
    # Treinar fold
    best_model, best_acc = train_fold(model, train_loader, val_loader, criterion, optimizer, patience= 15, min_delta=0.0)
    results.append(best_acc)
    
    # Salvar modelo se necessário
    torch.save(best_model, f'./models_salvos/best_model_MLP-500-classifier_fold{fold+1}.pt')

# 7. Resultados finais
print("\n=== Resultados ===")
print(f"Acurácia média: {np.mean(results):.4f} ± {np.std(results):.4f}")
print(f"Acurácias por fold: {results}")


=== Fold 1/3 ===
Epoch 1/1000000 - Train Loss: 3.2151 - Val Loss: 3.1185 - Val Acc: 0.3000
Epoch 2/1000000 - Train Loss: 2.9440 - Val Loss: 3.0022 - Val Acc: 0.3000
Epoch 3/1000000 - Train Loss: 2.7266 - Val Loss: 2.9592 - Val Acc: 0.3000
Epoch 4/1000000 - Train Loss: 2.5558 - Val Loss: 2.9082 - Val Acc: 0.2833
Epoch 5/1000000 - Train Loss: 2.4854 - Val Loss: 2.8574 - Val Acc: 0.2750
Epoch 6/1000000 - Train Loss: 2.4114 - Val Loss: 2.8275 - Val Acc: 0.2583
Epoch 7/1000000 - Train Loss: 2.3605 - Val Loss: 2.8103 - Val Acc: 0.2583
Epoch 8/1000000 - Train Loss: 2.3103 - Val Loss: 2.8024 - Val Acc: 0.2583
Epoch 9/1000000 - Train Loss: 2.3146 - Val Loss: 2.7910 - Val Acc: 0.2583
Epoch 10/1000000 - Train Loss: 2.2767 - Val Loss: 2.8022 - Val Acc: 0.2583
Epoch 11/1000000 - Train Loss: 2.2811 - Val Loss: 2.7998 - Val Acc: 0.2583
Epoch 12/1000000 - Train Loss: 2.2804 - Val Loss: 2.7882 - Val Acc: 0.2583
Epoch 13/1000000 - Train Loss: 2.2695 - Val Loss: 2.7803 - Val Acc: 0.2583
Epoch 14/1000000

In [98]:
model = MLP_500_classifier(22).to(device)
#carregando o melhor fold treinado
model.load_state_dict(torch.load('.//models_salvos/best_model_MLP-500-classifier_fold2.pt'))  # Substitua pelo caminho correto
model.eval()  # Modo de avaliação

MLP_500_classifier(
  (layer1): Linear(in_features=22, out_features=500, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=500, out_features=30, bias=True)
)

In [99]:
input = X[6]
input_tensor = torch.FloatTensor(input).to(device)

with torch.no_grad():
    logits = model(input_tensor)
    print(logits)
    print(logits.shape)
    probabilities = torch.softmax(logits, dim=0)  # Shape (1, 30)

tensor([ 1.6499e+00, -9.9736e-01,  8.9675e-01, -2.9397e+00, -5.3172e+00,
        -5.1559e+00,  1.4437e+00,  1.5168e+00, -7.5499e-01, -3.9873e+00,
        -2.8412e+00,  1.1642e-01,  4.0074e-01,  4.2145e-01,  1.4960e+00,
        -5.2945e+00, -5.9918e-01, -3.1868e+00, -4.3579e+00, -3.4973e+00,
        -2.9748e+00, -2.9152e-03, -3.0590e+00, -3.9453e+00, -2.7737e+00,
        -1.3477e+00, -4.1608e+00, -3.5662e+00, -4.0249e+00, -2.7908e+00])
torch.Size([30])


In [100]:
print(probabilities)

tensor([1.8440e-01, 1.3064e-02, 8.6832e-02, 1.8729e-03, 1.7378e-04, 2.0420e-04,
        1.5004e-01, 1.6142e-01, 1.6647e-02, 6.5697e-04, 2.0669e-03, 3.9792e-02,
        5.2877e-02, 5.3983e-02, 1.5810e-01, 1.7777e-04, 1.9454e-02, 1.4629e-03,
        4.5354e-04, 1.0725e-03, 1.8084e-03, 3.5315e-02, 1.6623e-03, 6.8521e-04,
        2.2111e-03, 9.2034e-03, 5.5234e-04, 1.0011e-03, 6.3276e-04, 2.1736e-03])


In [101]:
# Obter índices e valores das top-k probabilidades (ex.: top-5)
k = 3
top_k_probs, top_k_indices = torch.topk(probabilities, k=k, dim=0)

# Converter para numpy e exibir
top_k_probs = top_k_probs.cpu().numpy().flatten()
top_k_indices = top_k_indices.cpu().numpy().flatten()

print("Melhores ansatzes e suas probabilidades:")
for i, (idx, prob) in enumerate(zip(top_k_indices, top_k_probs)):
    print(f"{i+1}º: Ansatz {idx} - Probabilidade: {prob:.4f}")

Melhores ansatzes e suas probabilidades:
1º: Ansatz 0 - Probabilidade: 0.1844
2º: Ansatz 7 - Probabilidade: 0.1614
3º: Ansatz 14 - Probabilidade: 0.1581
